In [1]:
%%capture

!pip install datasets
!pip install evaluate

In [2]:
from google.colab import userdata
import os

sec_key = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = sec_key

We can write your own code loop, but hf provides a trainer class fo the process is more simple.


1. Load the model and
define the number of labels of the dataset.


2. Define the training arguments for you model (advanced) or it's possible use the defaults

3. The class don't evaluate (automaticaly), so you need to provide a metric for you class.

4. Create an objetc with all elements above.

5. Fine tune with `trainer.train()`.

In [3]:
from datasets import load_dataset


dataset = load_dataset("GonzaloA/fake_news")
dataset["train"][1]

README.md:   0%|          | 0.00/6.73k [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


dataset_infos.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/38.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/13.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24353 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8117 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8117 [00:00<?, ? examples/s]

{'Unnamed: 0': 1,
 'title': ' Trump’s Favorite News Channel Tries To Soothe His Battered Ego – Gets Taken To The Cleaners',
 'text': 'Yesterday, after the father of one of the UCLA players arrested in China failed to show Trump proper gratitude for getting his kid released, Trump, predictably, went to Twitter to grouse about it. He seems to expect to be worshiped for his help on this matter, but LaVar Ball wouldn t do it, so Trump tweeted:Now that the three basketball players are out of China and saved from years in jail, LaVar Ball, the father of LiAngelo, is unaccepting of what I did for his son and that shoplifting is no big deal. I should have left them in jail!  Donald J. Trump (@realDonaldTrump) November 19, 2017Fox News put that tweet into a meme portraying Trump as a strong, decisive leader and the UCLA basketball players as weaklings, because of course they did. Then they asked:  Do you agree with President Trump? Yes. Fox News seriously asked people whether they agree that Tr

In [4]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Map:   0%|          | 0/24353 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

Map:   0%|          | 0/8117 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 24353
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8117
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8117
    })
})

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [8]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,

)

In [12]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.031970,0.984000
2,No log,0.055061,0.984000
3,No log,0.061292,0.987000


TrainOutput(global_step=375, training_loss=0.08458770751953125, metrics={'train_runtime': 424.0312, 'train_samples_per_second': 7.075, 'train_steps_per_second': 0.884, 'total_flos': 789354427392000.0, 'train_loss': 0.08458770751953125, 'epoch': 3.0})

In [13]:
#Save model
trainer.save_model("content/sample_data")

In [14]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("/content/content/sample_data")

In [17]:
import torch

# Example text
texts = ["Yesterday, after the father of one of the UCLA players arrested in China failed to show Trump proper gratitude for getting his kid released, Trump, predictably, went to Twitter to grouse about it. He seems to expect to be worshiped for his help on this matter, but LaVar Ball wouldn t do it, so Trump tweeted:Now that the three basketball players are out of China and saved from years in jail, LaVar Ball, the father of LiAngelo, is unaccepting of what I did for his son and that shoplifting is no big deal. I should have left them in jail!  Donald J. Trump (@realDonaldTrump) November 19, 2017Fox News put that tweet into a meme portraying Trump as a strong, decisive leader and the UCLA basketball players as weaklings, because of course they did. Then they asked:  Do you agree with President Trump? Yes. Fox News seriously asked people whether they agree that Trump should have left them in jail because the father of one is refusing to show proper gratitude:Do you agree with President @realDonaldTrump? pic.twitter.com/p3VdoXPxPG  Fox News (@FoxNews) November 20, 2017And Twitter is just not having this at all:Nope."]

# Tokenize the inputs
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Move inputs to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = {key: val.to(device) for key, val in inputs.items()}

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

print(predictions)  # Output: tensor([1, 0]) (example labels: 1=positive, 0=negative)

tensor([0], device='cuda:0')
